In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
T = 4
Delta = 800
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]
print(cp_truth)

n = 20

[ 800 1600 2400]


In [6]:
path = '/content/drive/MyDrive/CPD_BT/experiment_random/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    beta_list = np.load(f)
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 3200, 20)

In [8]:
np.random.seed(0)

m_intervals = 50
grid_n = 200
gamma_list = [20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_wbs = np.zeros(B)
loc_error_wbs = np.zeros(B)
K_wbs = np.zeros(B)

cp_best_list = []
param_best_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    wbs_fit = wbs_cv_bt(m_intervals, grid_n, lam_list, gamma_list, smooth = 5, buffer = 5)
    res = wbs_fit.fit((X_train, Y_train), (X_test, Y_test))
    cp_best, cp_val, cusum_val, threshold_best, grid = res   
    run_time_wbs[b] = time.time() - start_time
    loc_error_wbs[b] = cp_distance(cp_best, cp_truth)
    K_wbs[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(threshold_best)

    print(b)

print('---------- wbs -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_wbs.mean(), run_time_wbs.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_wbs.std(), run_time_wbs.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_wbs < T - 1), sum(K_wbs == T - 1), sum(K_wbs > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- wbs -----------
avg loc error: 251.68, avg time: 133.72949260234833
std loc error: 219.90374621638443, std time: 14.67894917675527
K < K*: 0, K = K*: 40, K > K*: 60


In [9]:
loc_error_wbs

array([672.,  32., 368., 672.,   0.,   0., 272., 512., 464.,  16., 240.,
        96.,  16., 384.,  16., 496., 592., 624.,  16., 352., 608., 496.,
        16., 400.,  16.,   0.,   0., 480., 240., 256.,   0.,  32., 304.,
       416., 384., 544.,   0., 224., 512., 208., 400.,   0., 384., 528.,
       192.,  32., 384.,  16., 256., 272.,   0., 304.,   0., 544., 304.,
        16., 400., 144.,  48.,   0., 688.,   0., 656., 432., 448.,  48.,
       560.,   0.,   0.,   0., 240., 320., 144., 160.,  32., 464., 416.,
        16., 576.,  32., 336.,  32., 464., 496.,  16.,   0., 464., 400.,
        32., 336., 352., 432.,  16.,  16., 496.,   0.,  16., 448.,  32.,
       352.])

In [10]:
cp_best_list

[[816, 1600, 2048, 2400, 3072],
 [816, 1632, 2400],
 [432, 784, 1600, 2096, 2400, 2720],
 [512, 784, 1600, 2208, 2416, 2784, 3072],
 [800, 1600, 2400],
 [800, 1600, 2400],
 [800, 1072, 1600, 2400],
 [752, 1600, 2384, 2912],
 [336, 800, 1152, 1600, 2400, 2560],
 [816, 1600, 2400],
 [784, 1600, 1840, 2400],
 [800, 1600, 2496],
 [816, 1616, 2400],
 [816, 992, 1184, 1600, 2400],
 [784, 1600, 2400],
 [304, 800, 1600, 2400, 2752],
 [800, 1440, 1600, 2400, 2992],
 [784, 1152, 1600, 1776, 2416, 2768, 3024],
 [800, 1584, 2416],
 [496, 800, 1600, 2400, 2752],
 [192, 800, 1168, 1600, 2400, 2976],
 [800, 1600, 2384, 2896],
 [800, 1616, 2400],
 [784, 1600, 2032, 2336, 2800],
 [784, 1600, 2384],
 [800, 1600, 2400],
 [800, 1600, 2400],
 [752, 1600, 1808, 2400, 2880],
 [816, 1616, 2400, 2640],
 [800, 1584, 2464, 2656],
 [800, 1600, 2400],
 [816, 1568, 2432],
 [784, 1296, 1584, 2096, 2400],
 [800, 1312, 1600, 2032, 2368, 2816],
 [544, 800, 1008, 1328, 1600, 2400, 2784],
 [256, 800, 1104, 1600, 2400],
 

In [11]:
import pickle
with open(path + 'wbs_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, loc_error_wbs, run_time_wbs, K_wbs], f)